# Part 1 - Web Scraping and Data Wrangling

Import all the necessary libraries.

In [1]:
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests

Use beautifulsoup to perform web scraping.

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.content)

Perform Data Wrangling to get the desired dataframe.

In [3]:
table = soup.findUse beautifulsoup to perform web scraping('table', {"class": "wikitable sortable"})

In [4]:
rows = []
table_rows = table.findAll('tr')

In [5]:
for table_row in table_rows[1:]:
    table_cols = table_row.findAll('td')
    if not table_cols[1].text == 'Not assigned': 
        neighborhood = table_cols[2].text.replace('\n', '')
        borough = table_cols[1].text
        postcode = table_cols[0].text
        flag = 0
        if neighborhood == 'Not assigned':
            neighborhood = borough
        for idx, row in enumerate(rows):
            if postcode in row.values():
                rows[idx]['Neighborhood'] += ', ' + neighborhood
                flag = 1
                break
        if flag == 0:
            rows.append({
                'Postcode' : postcode,
                'Borough' : borough,
                'Neighborhood' : neighborhood
            })

Display our dataframe.

In [6]:
df = pd.DataFrame(rows)
print(df.head())
df.shape

            Borough                      Neighborhood Postcode
0        North York                         Parkwoods      M3A
1        North York                  Victoria Village      M4A
2  Downtown Toronto         Harbourfront, Regent Park      M5A
3        North York  Lawrence Heights, Lawrence Manor      M6A
4      Queen's Park                      Queen's Park      M7A


(103, 3)

# Part 2 - Geocoder and Adding Latitude, Longitude

Use geocoder to retrieve the latitude and longitude of each postcode.

In [7]:
import geocoder # import geocoder

# loop until you get the coordinates
for index, row in df.iterrows():
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(row['Postcode']))
    lat_lng_coords = g.latlng
    df.loc[df['Postcode'] == row['Postcode'], 'latitude'] = lat_lng_coords[0]
    df.loc[df['Postcode'] == row['Postcode'], 'longitude'] = lat_lng_coords[1]

Display our output.

In [8]:
df.head()

,Borough,Neighborhood,Postcode,latitude,longitude
0,North York,Parkwoods,M3A,43.752440,-79.329271
1,North York,Victoria Village,M4A,43.730421,-79.313320
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.655120,-79.362640
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.723125,-79.451589
4,Queen's Park,Queen's Park,M7A,43.661102,-79.391035


# Part 3 - Analysis and Clustering

Import the necessary libraries and retrieve the coordinates of Toronto, Ontario.

In [9]:
import folium
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


Display our coordinates using our previous dataframe on top of a map.

In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Insert the credentials needed to use the Foursquare API.

In [36]:
CLIENT_ID = 'ID here' # your Foursquare ID
CLIENT_SECRET = 'Secret here' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100
radius = 500

Your credentails:
CLIENT_ID: ID here
CLIENT_SECRET:Secret here


Only include rows with the borough containing 'Toronto'

In [12]:
toronto_df = df[df['Borough'].str.contains('Toronto')].reset_index()
toronto_df.head()

,index,Borough,Neighborhood,Postcode,latitude,longitude
0,2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.655120,-79.362640
1,9,Downtown Toronto,"Ryerson, Garden District",M5B,43.657363,-79.378180
2,15,Downtown Toronto,St. James Town,M5C,43.651210,-79.375481
3,19,East Toronto,The Beaches,M4E,43.676845,-79.295225
4,20,Downtown Toronto,Berczy Park,M5E,43.645160,-79.373675


Display the new dataframe's coordinates on top of the new dataframe.

In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(
    toronto_df['latitude'],
    toronto_df['longitude'],
    toronto_df['Borough'],
    toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)Only include rows with the borough containing 'Toronto'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Create the url and the function to get nearby venues.

In [14]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Call the newly created function passing in our dataframe's neighborhood, latitude and longitude columns.

In [16]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['latitude'],
                                   longitudes=toronto_df['longitude'],
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

Display the newly created dataframe containing the nearby venue of each entry.

In [17]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Harbourfront, Regent Park",43.65512,-79.36264,Cocina Economica,43.654959,-79.365657,Mexican Restaurant
4,"Harbourfront, Regent Park",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


Filter by neighborhood.

In [18]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,62,62,62,62,62,62
"Brockton, Exhibition Place, Parkdale Village",70,70,70,70,70,70
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",69,69,69,69,69,69
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,98,98,98,98,98,98
"Chinatown, Grange Park, Kensington Market",96,96,96,96,96,96
Christie,10,10,10,10,10,10


Perform one hot encoding for each venue category.

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group by neighborhood and just retrieve the mean of each venue category.

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.0,0.03,0.0,0.010000,0.000000,0.030000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.000000,0.01,0.0
1,Berczy Park,0.000000,0.0,0.00,0.0,0.016129,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.016129,0.0,0.000000,0.00,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0,0.00,0.0,0.028571,0.014286,0.000000,0.0,0.014286,...,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.014286,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0,0.03,0.0,0.010000,0.000000,0.020000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.000000,0.00,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.014493,0.0,0.00,0.0,0.000000,0.000000,0.014493,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.014493,0.0,0.000000,0.00,0.0


Display the top 5 nearby venues per entry in the dataframe.

In [21]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.07
2                Hotel  0.05
3  Japanese Restaurant  0.03
4         Burger Joint  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.10
1    Restaurant  0.05
2        Bakery  0.05
3  Cocktail Bar  0.05
4   Cheese Shop  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.09
1  Furniture / Home Store  0.06
2                    Café  0.06
3              Restaurant  0.06
4          Sandwich Place  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.09
1          Bar  0.06
2         Café  0.05
3   Steakhouse  0.04
4        Hotel  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Coffee Shop  0.12
1  Italian Restaurant  0.07
2    

Create a function to return the most common venues in order to create a new dataframe.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Burger Joint,Asian Restaurant,Steakhouse,Bar,Deli / Bodega,Restaurant,Gastropub
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Farmers Market,Lounge,Steakhouse,Seafood Restaurant,Breakfast Spot,Cheese Shop
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Furniture / Home Store,Restaurant,Sandwich Place,Hotel,Beer Bar,Supermarket,Bar,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Bar,Café,Hotel,Steakhouse,American Restaurant,Pub,Pizza Place,Italian Restaurant,Japanese Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Bar,Restaurant,Café,Sandwich Place,Speakeasy,Park,Gym / Fitness Center,Pub


Perform k-means clustering on our new dataframe with k being 5.

In [24]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Add the cluster label on the dataframe.

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,index,Borough,Neighborhood,Postcode,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.655120,-79.362640,0,Coffee Shop,Restaurant,Breakfast Spot,Italian Restaurant,Gym / Fitness Center,Mexican Restaurant,Health Food Store,Spa,Food Truck,Electronics Store
1,9,Downtown Toronto,"Ryerson, Garden District",M5B,43.657363,-79.378180,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Tea Room,Plaza,Ramen Restaurant,Diner
2,15,Downtown Toronto,St. James Town,M5C,43.651210,-79.375481,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Hotel,Cosmetics Shop,Breakfast Spot,Clothing Store,Cocktail Bar,Bakery
3,19,East Toronto,The Beaches,M4E,43.676845,-79.295225,0,Health Food Store,Trail,Pub,Wings Joint,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,20,Downtown Toronto,Berczy Park,M5E,43.645160,-79.373675,0,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Farmers Market,Lounge,Steakhouse,Seafood Restaurant,Breakfast Spot,Cheese Shop


Display the map using the new dataframe with the color of the points signifying the cluster membership.

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Display our first cluster.

Cluster Label - Diners

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.362640,0,Coffee Shop,Restaurant,Breakfast Spot,Italian Restaurant,Gym / Fitness Center,Mexican Restaurant,Health Food Store,Spa,Food Truck,Electronics Store
1,Downtown Toronto,-79.378180,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Tea Room,Plaza,Ramen Restaurant,Diner
2,Downtown Toronto,-79.375481,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Hotel,Cosmetics Shop,Breakfast Spot,Clothing Store,Cocktail Bar,Bakery
3,East Toronto,-79.295225,0,Health Food Store,Trail,Pub,Wings Joint,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,Downtown Toronto,-79.373675,0,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Farmers Market,Lounge,Steakhouse,Seafood Restaurant,Breakfast Spot,Cheese Shop
5,Downtown Toronto,-79.384930,0,Coffee Shop,Clothing Store,Cosmetics Shop,Tea Room,Plaza,Middle Eastern Restaurant,Sushi Restaurant,Hotel,Lingerie Store,Sandwich Place
6,Downtown Toronto,-79.420710,0,Café,Grocery Store,Athletics & Sports,Italian Restaurant,Baby Store,Coffee Shop,Playground,Fish & Chips Shop,Fish Market,Ethiopian Restaurant
7,Downtown Toronto,-79.382503,0,Coffee Shop,Café,Hotel,Burger Joint,Asian Restaurant,Steakhouse,Bar,Deli / Bodega,Restaurant,Gastropub
8,West Toronto,-79.438705,0,Park,Pharmacy,Bus Line,Brazilian Restaurant,Furniture / Home Store,Middle Eastern Restaurant,Fast Food Restaurant,Bar,Bank,Bakery
10,West Toronto,-79.417742,0,Bar,Coffee Shop,Restaurant,Cocktail Bar,Asian Restaurant,French Restaurant,Vietnamese Restaurant,Pizza Place,New American Restaurant,Bakery


Display our second cluster.

Cluster Label - Leisure

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,-79.414405,1,Park,Wings Joint,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
21,West Toronto,-79.462874,1,Park,Wings Joint,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Display our third cluster.

Cluster Label - Home Service

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,-79.41912,2,Home Service,Cuban Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Display our fourth cluster.

Cluster Label - Harbor

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,-79.391507,3,Harbor / Marina,Music Venue,Café,Wings Joint,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


Display our fifth cluster.

Cluster Label - Transportation

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,-79.387085,4,Bus Line,Swim School,Wings Joint,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


# Conclusion

As you can see, most of the entries in our dataframe were placed in the first cluster because of the fact that most common venues for those places were coffee shops, cafes, restaurants and diners. We can conclude that most of the neighborhoods in Toronto have restaurants, coffee shops, cafes and diners as their most common venue which is why it is common for a neighborhood in Toronto to belong on the first cluster.